In [2]:
import os
import numpy as np
import matplotlib
import torch
# matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
import torch
import copy
from tqdm.auto import tqdm
# from torch.utils.tensorboard import SummaryWriter
work_dir = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
lines = open("data/Data.csv", "r", encoding="utf-8").read().split("\n")

In [4]:
for i, clo_name in enumerate(lines[0].split(",")):
    print(i, clo_name)

0 "PART_ID"
1 "STATIS_ID"
2 "INFO_FLAG"
3 "LJPYM"
4 "STN_NAME"
5 "STN_CODE"
6 "STN_XH"
7 "DAODA_TIME"
8 "CHUFA_TIME"
9 "TIME_FLAG"
10 "ARR_NAME"
11 "DPT_NAME"
12 "TRAIN_ID"
13 "ARRIVAL0"
14 "DEPART0"
15 "XH"
16 "KY_STN_NAME"
17 "ARR_DELAY_FLAG"
18 "ARR_DELAY"
19 "DPT_DELAY_FLAG"
20 "DPT_DELAY"
21 "TRACK"
22 "TRACK_NAME"
23 "FORE_DELAY"
24 "SC_RENSHU"
25 "XC_RENSHU"
26 "FORE_ARR_DELAY"
27 "FORE_DPT_DELAY"
28 "ARRIVAL1"
29 "DEPART1"
30 "ARR_DELAY1"
31 "DPT_DELAY1"
32 "STN_FLAG"
33 "RW_FLAG"
34 "ARR_ER"
35 "DPT_ER"
36 "XM"
37 "EP_FLAG"
38 "DIS_FLAG"
39 "SECTION_NAME"
40 "SECTION_CHUFA_TIME"
41 "SECTION_DPT_DELAY"
42 "SW_DATE"
43 "SWH"
44 "BEFORE_ID"
45 "BEFORE_DPT_DELAY"


In [5]:
small_dataset = []
station_set = set()
train_set = set()
for i, line in tqdm(enumerate(lines[1:])):
    if line:
        tmp = line.split(",")
        STN_NAME = tmp[4].strip()
        STN_XH = tmp[6].strip()
        ARR_NAME = tmp[10].strip()
        ARR_DELAY = tmp[18].strip()
        DPT_DELAY = tmp[20].strip()
        item = {
            "line_id": i,
            "STN_NAME": STN_NAME,
            "STN_XH": int(eval(STN_XH)),
            "ARR_NAME": ARR_NAME,
            "ARR_DELAY": int(ARR_DELAY),
            "DPT_DELAY": int(DPT_DELAY),
        }
        small_dataset.append(item)
        station_set.add(STN_NAME)
        train_set.add(ARR_NAME)


0it [00:00, ?it/s]

In [6]:
print(line)

In [7]:
station_2_id = {station_name: i + 1 for i, station_name in enumerate(station_set)}
train_2_id = {train_name: i + 1 for i, train_name in enumerate(train_set)}


In [8]:
last_xh = -1
last_station = ""
last_train = ""
history_data_list = []
all_train_data = []
label_list = []
max_seq_len = 0
for item in tqdm(small_dataset):
    if last_train == "" or item["ARR_NAME"] != last_train:
        history_data_list = []
    # while last_xh != -1 and last_xh < item["STN_XH"]:
    #     last_xh += 1
    #     history_data_list.append(np.array([station_2_id[last_station], last_xh, train_2_id[last_train], -1, -1]))
    history_data_list.append(np.array([station_2_id[item["STN_NAME"]], item["STN_XH"], train_2_id[item["ARR_NAME"]], 0, 0], dtype = np.float32))
    if len(history_data_list) != 1:
        one_train_data_numpy = np.stack(history_data_list)
        max_seq_len = max(one_train_data_numpy.shape[0], max_seq_len)
        all_train_data.append(one_train_data_numpy)
        label_list.append(np.array([item["ARR_DELAY"], item["DPT_DELAY"]], dtype = np.float32))
    history_data_list[-1] = np.array([station_2_id[item["STN_NAME"]], item["STN_XH"], train_2_id[item["ARR_NAME"]], item["ARR_DELAY"], item["DPT_DELAY"]], dtype = np.float32)
    last_xh = item["STN_XH"]
    last_station = item["STN_NAME"]
    last_train = item["ARR_NAME"]

  0%|          | 0/2694800 [00:00<?, ?it/s]

In [8]:
all_train_data

[array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,  -5,  -5],
        [230,   2, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,  -5,  -5],
        [230,   2, 152,  -6,  -6],
        [130,   5, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,  -5,  -5],
        [230,   2, 152,  -6,  -6],
        [130,   5, 152,  -6,  -6],
        [ 79,   5, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        

In [9]:
max_seq_len

141

In [10]:
padded_all_train_data = []
for one_train_data_numpy in tqdm(all_train_data):
    if one_train_data_numpy.shape[0] < max_seq_len:
        padded_all_train_data.append(np.concatenate((one_train_data_numpy, np.zeros((max_seq_len - one_train_data_numpy.shape[0], one_train_data_numpy.shape[1]), dtype = np.float32)), axis=0))
    else:
        padded_all_train_data.append(one_train_data_numpy)

  0%|          | 0/2187535 [00:00<?, ?it/s]

In [11]:
del lines
del all_train_data

In [12]:
feature_numpy = np.stack(padded_all_train_data)
label_numpy = np.stack(label_list)
np.save(f'{work_dir}data/feature_tensor.npy', feature_numpy)
np.save(f'{work_dir}data/label_tensor.npy', label_numpy)
np.save(f'{work_dir}data/station_2_id.npy', station_2_id)
np.save(f'{work_dir}data/train_2_id.npy', train_2_id)

In [14]:
feature_numpy = np.load(f'{work_dir}data/feature_tensor.npy')
label_numpy = np.load(f'{work_dir}data/label_tensor.npy')
station_2_id = np.load(f'{work_dir}data/station_2_id.npy', allow_pickle=True)
train_2_id = np.load(f'{work_dir}data/train_2_id.npy', allow_pickle=True)
feature_tensor = torch.from_numpy(feature_numpy)
label_tensor = torch.from_numpy(label_numpy)
(feature_tensor.shape, label_tensor.shape)

(torch.Size([2187535, 141, 5]), torch.Size([2187535, 2]))

In [ ]:
type(feature_tensor)

In [15]:
train_dataset = torch.utils.data.TensorDataset(feature_tensor, label_tensor)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, sampler=None)

In [16]:
class Train_LSTM_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.batch_first = True
        self.lstm = torch.nn.LSTM(feature_tensor.shape[-1], 256, 2, batch_first=self.batch_first) # (input_size, hidden_size, num_layers)
        self.linear1 = nn.Linear(256, 512)
        self.linear2 = nn.Linear(512, 2)

    def forward(self, input_tensor, seq_lens=[]):
        # input_tensor shape: batch_size*time_step*dim , seq_lens: (batch_size,)  when batch_first = True
        # total_length = input_tensor.size(1) if self.batch_first else input_tensor.size(0)
        # x_packed = pack_padded_sequence(input_tensor, seq_lens, batch_first=self.batch_first, enforce_sorted=False)
        y_lstm, hidden = self.lstm(input_tensor)
        # y_padded, length = pad_packed_sequence(y_lstm, batch_first=self.batch_first, total_length=total_length)
        # print(y_lstm)
        y_lstm, out_len = torch.nn.utils.rnn.pad_packed_sequence(y_lstm, batch_first=True)
        # print(y_lstm)

        res = y_lstm[:,-1]
        res = self.linear1(res)
        res = self.linear2(res)

        return res

In [18]:
model = Train_LSTM_Model()
model.train()
model.to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-7)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(epoch+1))

min_loss = 10000.0000
epochs = 400
for i in tqdm(range(epochs)):
    for x, y in train_data_loader:
        length = [torch.nonzero(single_data[:,0]==0)[0].item() if torch.nonzero(single_data[:,0]==0).shape[0] != 0 else single_data.shape[0] for single_data in x]
        x = torch.nn.utils.rnn.pack_padded_sequence(x, length, batch_first=True, enforce_sorted=False)
        x = x.to(device)
        optimizer.zero_grad()

        y_pred = model(x).to('cpu')
        # print(y_pred.shape)

        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()
    if i % 1 == 0:
        print(f'epoch: {i} loss: {single_loss.item(): .4f}\tlr: {optimizer.param_groups[0]["lr"]: .10f}')
    if i % 2 == 0:
        scheduler.step()
    if single_loss.item() < min_loss:
        min_loss = single_loss.item()
        best_model_path = f'{work_dir}model/Train_LSTM_Model-learn_rate_1e4-mse_loss-adam-epoch_{i}-loss_{single_loss}-last_lr_{optimizer.param_groups[0]["lr"]: .10f}.chpk'
        torch.save(model, best_model_path)

  0%|          | 0/400 [00:00<?, ?it/s]

epoch: 0 loss:  845.7653	lr:  0.0001000000
epoch: 1 loss:  10.7334	lr:  0.0000500000
epoch: 2 loss:  12.2328	lr:  0.0000500000
epoch: 3 loss:  8.7842	lr:  0.0000500000
epoch: 4 loss:  15.6962	lr:  0.0000500000
epoch: 5 loss:  14.6526	lr:  0.0000500000
epoch: 6 loss:  7.9546	lr:  0.0000500000


KeyboardInterrupt: 

In [27]:
model = torch.load("model/Train_LSTM_Model-learn_rate_1e2-mse_loss-adam-epoch_21-loss_2.166508913040161-last_lr_ 0.0050000000.chpk")
model.eval()
model.to(device)
arr_correct_cum = 0
dep_correct_cum = 0
total_num = 0

for x, y in tqdm(train_data_loader):
    length = [torch.nonzero(single_data[:,0]==0)[0].item() if torch.nonzero(single_data[:,0]==0).shape[0] != 0 else single_data.shape[0] for single_data in x]
    x = torch.nn.utils.rnn.pack_padded_sequence(x, length, batch_first=True, enforce_sorted=False)
    x = x.to(device)
    optimizer.zero_grad()

    y_pred = model(x).to('cpu')
    arr_correct_cum += sum(torch.round(y_pred[:,0]) == y[:,0])
    dep_correct_cum += sum(torch.round(y_pred[:,1]) == y[:,1])
    total_num += y_pred.shape[0]
    single_loss = loss_function(y_pred, y)

    # print(f'test loss: {single_loss.item(): .4f}')
print(arr_correct_cum, dep_correct_cum, total_num)
print(arr_correct_cum / total_num, dep_correct_cum / total_num)

  0%|          | 0/17091 [00:00<?, ?it/s]

tensor(179711) tensor(162051) 2187535
tensor(0.0822) tensor(0.0741)


In [23]:
x

PackedSequence(data=tensor([[ 2.1700e+02,  0.0000e+00,  1.5000e+01, -1.0000e+00,  0.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  3.5300e+02,  2.7000e+01,  2.7000e+01],
        [ 4.1000e+01,  0.0000e+00,  1.8900e+02,  0.0000e+00,  0.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  1.8000e+02, -1.0000e+00, -1.0000e+00],
        [ 4.1000e+01,  0.0000e+00,  1.0600e+02,  0.0000e+00,  0.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  1.1610e+03,  0.0000e+00,  0.0000e+00],
        [ 4.1000e+01,  2.0000e+00,  1.9200e+02, -3.0000e+00,  0.0000e+00],
        [ 1.1100e+02,  1.0000e+00,  6.7800e+02,  0.0000e+00,  0.0000e+00],
        [ 4.1000e+01,  0.0000e+00,  8.4100e+02,  1.0000e+00,  1.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  2.0700e+02, -1.0000e+00, -1.0000e+00],
        [ 2.5800e+02,  2.0000e+00,  6.2200e+02, -2.0000e+00, -2.0000e+00],
        [ 2.0600e+02,  3.0000e+00,  2.3700e+02, -1.0000e+00, -1.0000e+00],
        [ 2.0600e+02,  3.0000e+00,  1.4500e+02,  2.0000e+00,  2.0000e+00],
     

In [28]:
y_pred

tensor([[1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [0.4807, 0.4377],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530]], grad_fn=<ToCopyBackward0>)

In [25]:
torch.round(y_pred)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [0., 0.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]], grad_fn=<RoundBackward0>)

In [26]:
y

tensor([[ 1.,  1.],
        [-4., -4.],
        [ 2.,  2.],
        [ 0.,  0.],
        [-1., -1.],
        [ 1.,  1.],
        [68., 68.],
        [-1., -1.],
        [ 0.,  0.],
        [ 2.,  2.],
        [ 5.,  5.],
        [ 3.,  3.],
        [-1., -2.],
        [-2., -2.],
        [ 0.,  0.]])

In [34]:
torch.nonzero(x[1][:,0]==0)[0].item()

6

In [1]:
import pandas as pd

df = pd.read_csv('data/Data.csv')

print(df.to_string())

MemoryError: 

In [ ]:
print(df.head())

